In [145]:

import sqlalchemy as db
import pandas as pd
from sqlalchemy import Table,Column,insert,select,desc,update
from sqlalchemy.sql.expression import select,func
from sqlalchemy.orm import Session


In [105]:
 
engine = db.create_engine('sqlite:///orm.db')
conn = engine.connect()
metadata = db.MetaData()

Track = Table('Track',metadata,
Column('id',db.Integer(),autoincrement=True,primary_key=True,unique=True,nullable=True),
Column('title',db.String(128),unique=True,nullable=True),
Column('album_id',db.Integer()))

Track.drop(engine,checkfirst=True)
Track.create(engine,checkfirst=True)

Album = Table('Album',metadata,
Column('id',db.Integer(),autoincrement=True,primary_key=True,unique=True,nullable=True),
Column('name',db.Text(),unique=True),
Column('artist_id',db.Integer(),nullable=True))

Album.drop(engine,checkfirst=True)
Album.create(engine,checkfirst=True)

Artist = Table('Artist',metadata,
Column('id',db.Integer(),primary_key=True,autoincrement=True,nullable=True),
Column('Artist_name',db.Text(),unique=True,nullable=True))

Artist.drop(engine,checkfirst=True)
Artist.create(engine,checkfirst=True)

# Inserting values to the data bases

query = insert(Album)
val = [{'name':'Album01','artist_id':'1'},{'name':'Album02','artist_id':'2'},{'name':'Album03','artist_id':'1'},{'name':'Album04','artist_id':'2'}]
r = conn.execute(query,val)

query = insert(Track)
val = [{'title':'Title01','album_id':'1'},{'title':'Title02','album_id':'1'},{'title':'Title03','album_id':'4'},{'title':'Title04','album_id':'2'}
       ,{'title':'Title05','album_id':'3'},{'title':'Title06','album_id':'2'},{'title':'Title07','album_id':'4'},{'title':'Title08','album_id':'1'}]
r = conn.execute(query,val)

query = insert(Artist)
val = [{'Artist_name':'name 1'},{'Artist_name':'name 2'}]
r = conn.execute(query,val)



In [106]:
query = select(Track.columns.title,Album.columns.name).join(Album,Track.columns.album_id==Album.columns.id)
res = Session(bind=engine).execute(query)

for i in res:
    print(i)

('Title01', 'Album01')
('Title02', 'Album01')
('Title03', 'Album04')
('Title04', 'Album02')
('Title05', 'Album03')
('Title06', 'Album02')
('Title07', 'Album04')
('Title08', 'Album01')


In [107]:
# ordering by descending order
query = select(Track.columns.title,Album.columns.name,Artist.columns.Artist_name).\
    order_by(desc(Track.columns.id)).join(Album,Track.columns.album_id==Album.columns.id).join(Artist,Album.columns.artist_id==Artist.columns.id)
res = Session(bind=engine).execute(query)   # we can add \ when we don't want the line to be lenghty
res.fetchone() # fetchone / fetchall() print all the stuffs no need to print those separately
# for row in res:
#     print(row)

('Title08', 'Album01', 'name 1')

In [108]:

# ordering rows randomly  # limit

query = select(Track.columns.title,Album.columns.name,Artist.columns.Artist_name).\
    order_by(func.random()).limit(1).join(Album,Track.columns.album_id==Album.columns.id).join(Artist,Album.columns.artist_id==Artist.columns.id)
res = Session(bind=engine).execute(query)   # we can add \ when we don't want the line to be lenghty

# res.fetchone()    # fetchone / fetchall() print all the stuffs no need to print those separately


for row in res:
    print(row)

('Title07', 'Album04', 'name 2')


In [144]:
# res = conn.execute(insert(Track).values(title = 'title 09'))



# how to find rows where column component is NULL

res = Session(bind=engine).execute(select(Track).where(Track.columns.album_id == None))
item = res.fetchall()[0] # you have to store res.fetchall() as list or fetchone()  as tuple
print(item[0],item[1])

9 title 09


In [110]:
session = Session(bind=engine)

res = session.execute(select(Track).order_by(func.random()).limit(1).where(Track.columns.album_id == 1))
res.fetchone()[1]

'Title08'

In [148]:
# updating values 

query = update(Track).where(Track.columns.title == 'title 09').values(title = 'title 10')
r = conn.execute(query)

In [149]:
query = update(Track,Track.columns.id == 9,{'title':'title 9'})
r = conn.execute(query)

In [150]:
# deleting row from table

r = conn.execute(db.delete(Track,Track.columns.id == 9))


In [157]:
#distinct rows
query = select(db.distinct(Track.columns.album_id))
rows = Session(bind=engine).execute(query)


for row in rows:
    print(row)

(1,)
(4,)
(2,)
(3,)
